In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [2]:
df = pd.read_excel("TestData1.xlsx")

In [3]:
df.head()

,Contract,WP_#,Emp_Notes_ID,Band,Band_Wt,Status,Confidence,NCE_Start_Date,NCE_End_Date,Transition_Start_Date,Transition_End_Date,Billing_Start_Date,Billing_End_Date,Premium
0,B04331,T&M,Praveen K Haarkara/India/IBM,7B,7.5,Inactive,NaN,NaN,NaN,NaN,NaN,2016-12-03,2017-03-15,Standard
1,B04331,T&M,Srikanth Merugu3/India/IBM,6B,6.5,Inactive,NaN,NaN,NaN,NaN,NaN,2016-12-05,2017-03-15,Standard
2,B04349,T&M,Abhijit Murade/India/IBM,7A,7.0,Active,NaN,NaN,NaN,NaN,NaN,2016-12-03,NaT,Standard
3,B04358,T&M,Avinash Singh10/India/IBM,6B,6.5,Active,Identified,2017-03-27 00:00:00,2017-04-14 00:00:00,NaN,NaN,2017-04-17,NaT,Standard
4,B04358,T&M,Teradata 8,6B,6.5,Active,Medium,2017-04-03 00:00:00,2017-04-14 00:00:00,NaN,NaN,2017-04-17,NaT,Standard


In [4]:
df.describe()

,Band_Wt
count,25.00000
mean,6.74000
std,0.49202
min,6.00000
25%,6.50000
50%,6.50000
75%,7.00000
max,7.50000


In [5]:
#Let's understand the dimensions of the data in the dataframe
df.shape

(25, 14)

In [6]:
len(df.index)

25

In [7]:
list(df.columns.values)

['Contract',
 'WP_#',
 'Emp_Notes_ID',
 'Band',
 'Band_Wt',
 'Status',
 'Confidence',
 'NCE_Start_Date',
 'NCE_End_Date',
 'Transition_Start_Date',
 'Transition_End_Date',
 'Billing_Start_Date',
 'Billing_End_Date',
 'Premium']

In [8]:
df[df.Band == '7A']

,Contract,WP_#,Emp_Notes_ID,Band,Band_Wt,Status,Confidence,NCE_Start_Date,NCE_End_Date,Transition_Start_Date,Transition_End_Date,Billing_Start_Date,Billing_End_Date,Premium
2,B04349,T&M,Abhijit Murade/India/IBM,7A,7.0,Active,NaN,NaN,NaN,NaN,NaN,2016-12-03,NaT,Standard
13,B04358,T&M,Tushar Ab Initio TBD 1,7A,7.0,Forecast,Low,NaN,NaN,NaN,NaN,2017-04-15,2017-12-31,Premium
18,B04358,T&M,Prashant Garg12/India/IBM,7A,7.0,Forecast,Identified,NaN,NaN,NaN,NaN,2017-05-02,NaT,Premium


In [9]:
df[df.duplicated(['Emp_Notes_ID','Status'],keep=False)]

,Contract,WP_#,Emp_Notes_ID,Band,Band_Wt,Status,Confidence,NCE_Start_Date,NCE_End_Date,Transition_Start_Date,Transition_End_Date,Billing_Start_Date,Billing_End_Date,Premium


In [10]:
now =  datetime.now()
df[((df['Billing_End_Date'].isnull()) | (df['Billing_End_Date'] > now)) & (df['Status'] == 'Inactive')]

,Contract,WP_#,Emp_Notes_ID,Band,Band_Wt,Status,Confidence,NCE_Start_Date,NCE_End_Date,Transition_Start_Date,Transition_End_Date,Billing_Start_Date,Billing_End_Date,Premium


In [11]:
df['Count_on'] = 1
df['Prem_pct'] = (df.groupby('Premium')['Count_on'].transform('sum') / df.Count_on.sum())*100

In [12]:
def getDaysForMonth(startDate, endDate, forMonth):
    thisYr = 2017
    first_date = pd.Series({1:date(thisYr,1,1), 2:date(thisYr,2,1), 3:date(thisYr,3,1), 4:date(thisYr,4,1), 5:date(thisYr,5,1), 6:date(thisYr,6,1), 7:date(thisYr,7,1), 8:date(thisYr,8,1), 9:date(thisYr,9,1), 10:date(thisYr,10,1), 11:date(thisYr,11,1), 12:date(thisYr,12,1)})
    last_date = pd.Series({1:date(thisYr,1,31), 2:date(thisYr,2,28), 3:date(thisYr,3,31), 4:date(thisYr,4,30), 5:date(thisYr,5,31), 6:date(thisYr,6,30), 7:date(thisYr,7,31), 8:date(thisYr,8,31), 9:date(thisYr,9,30), 10:date(thisYr,10,31), 11:date(thisYr,11,30), 12:date(thisYr,12,31)})
    #if pd.isnull(startDate):
    #    return 0
    if startDate == date.min:
        return 0
    elif startDate > last_date[forMonth]:
        return 0
    elif startDate < first_date[forMonth]:
        startDate = first_date[forMonth]
    
    #if pd.isnull(endDate):
    #    endDate = date(thisYr,12,31)
    if endDate == date.max:
        endDate = last_date[forMonth]
    elif endDate < first_date[forMonth]:
        return 0
    elif endDate > last_date[forMonth]:
        endDate = last_date[forMonth]    
    
    delta = np.busday_count(startDate, endDate)
    return delta

def getRevenue (startDate, endDate):
    revenueByMonth = {}
    #Define the billing rate for each month
    billingRate = {1:121, 2:121, 3:121, 4:121, 5:121, 6:121, 7:121, 8:121, 9:121, 10:121, 11:121, 12:121}
    for month in range(1,13):
        revenueByMonth[month] = getDaysForMonth(startDate, endDate, month) * billingRate[month]
    return revenueByMonth

def calcRevenue (df1, startDate, endDate):
    #Define the billing rate for each month
    billingRate = {1:121, 2:121, 3:121, 4:121, 5:121, 6:121, 7:121, 8:121, 9:121, 10:121, 11:121, 12:121}
    for month in range(1,13):
        df1["Rev_Mon_"+ str(month)] = getDaysForMonth(startDate, endDate, month) * billingRate[month]
    return df1
    

In [13]:
getDaysForMonth(date(2017,2,15),date(2017,2,25),3)

0

In [14]:
getDaysForMonth(date(2017,2,15),date(2017,3,25),3)

18

In [15]:
getDaysForMonth(date(2017,3,15),date(2017,3,25),3)

8

In [16]:
getDaysForMonth(date(2017,3,15),date(2017,4,25),3)

12

In [17]:
getDaysForMonth(date(2017,4,15),date(2017,4,25),3)

0

In [18]:
getRevenue(date(2017,2,15),date(2017,8,25))

{1: 0,
 2: 1089,
 3: 2662,
 4: 2420,
 5: 2662,
 6: 2541,
 7: 2420,
 8: 2178,
 9: 0,
 10: 0,
 11: 0,
 12: 0}

In [19]:
df2 = pd.DataFrame()
for index, row in df.iterrows():
    nceStart = row['NCE_Start_Date']
    nceEnd = row['NCE_End_Date']
    transitionStart = row['Transition_Start_Date']
    transitionEnd = row['Transition_End_Date']
    billingStart = row['Billing_Start_Date']
    billingEnd = row['Billing_End_Date']
    
    if pd.isnull(billingEnd): #Very important technique of finding null -- pd.isnull(xxx)
        billingEnd = datetime(2017,12,31,0,0,0)
        
    revSer = getRevenue(billingStart.date(), billingEnd.date())
    df2 = df2.append([revSer],index) #Just a df2.append([..]) does not work. Has to be df2 = df2...

df3 = df.join(df2)
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('calculations.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df3.to_excel(writer, sheet_name='Revenue')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
print("Done")

Done


In [20]:
#df["Jan_Rev"] = getDaysForMonth(df.Billing_Start_Date.date(),df.Billing_End_Date.date(),1)
df['Billing_End_Date'].fillna(date(2017,12,31),inplace=True)
df.assign(Jan_Billing_Days = lambda df: getDaysForMonth(df.Billing_Start_Date,df.Billing_End_Date,1))

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 00:00:00

In [ ]:
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0/values[i]
    return output

values = np.random.randint(1,10,size=5)
compute_reciprocals(values)

In [ ]:
big_array = np.random.randint(1,100,size = 1000000)
%timeit compute_reciprocals(big_array)

In [ ]:
%timeit (1.0/big_array)